In [42]:
import pandas as pd
data = pd.read_csv('train.csv')

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as datum
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [44]:
data.head()

,Unnamed: 0,DATE,LATITUDE_A,LONGITUDE_A,ELEVATION_A,PRCP_A,SNWD_A,TMAX_A,TMIN_A,TAVG_A,...,LONGITUDE_C,ELEVATION_C,PRCP_C,SNWD_C,TMAX_C,TMIN_C,TAVG_C,LATITUDE,LONGITUDE,TAVG
0,0,01-11-1979,63.88,77.52,128,NaN,0.0,17.6,NaN,12.96,...,78.93,64,0.0,0.00,18.7,NaN,NaN,63.933,78.733,9.3
1,1,13-11-1979,63.88,77.52,128,NaN,0.0,8.8,NaN,0.00,...,78.93,64,0.0,0.00,11.0,-3.8,2.04,63.933,78.733,2.7
2,2,05-02-1980,63.88,77.52,128,NaN,0.0,-9.9,NaN,-17.76,...,78.93,64,0.0,0.00,NaN,NaN,-16.56,63.933,78.733,-16.0
3,3,23-12-1979,63.88,77.52,128,0.0,0.0,NaN,-4.30,-4.44,...,78.93,64,0.0,0.00,0.0,NaN,-2.64,63.933,78.733,-3.2
4,4,07-06-1977,63.88,77.52,128,NaN,0.0,39.6,15.48,NaN,...,78.93,64,NaN,3.82,38.5,17.1,33.00,63.933,78.733,27.3


In [45]:
cty = {'A': {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0},
       'B': {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0},
       'C': {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}}
# cty = {'A': 0, 'B': 0, 'C': 0}
# fld = {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}
# cty = {i: fld for i, x in cty.items()}
# print(cty)
for x, dat in enumerate(data.itertuples()):
    for key, val in cty.items():
        for ind, tvl in val.items():
            rem = list(val.keys())
            rem.remove(ind)
            if pd.isna(getattr(dat, f'{ind}_{key}')):
                v1, v2 = getattr(dat, f'{rem[0]}_{key}'), getattr(dat, f'{rem[1]}_{key}')
                if not pd.isna(v1) and not pd.isna(v2) and ind != 'PRCP' and ind != 'SNWD':
                    data.loc[x, f'{ind}_{key}'] = ((2 * v2) - v1) if ind in ['TMAX', 'TMIN'] else ((v1 + v2) / 2)
                    cty[key][ind] += 1
                else:
                    rec = list(cty.keys())
                    rec.remove(key)
                    for ciy in rec:
                        if not pd.isna(getattr(dat, f'{ind}_{ciy}')):
                            data.loc[x, f'{ind}_{key}'] = float(getattr(dat, f'{ind}_{ciy}'))
                            cty[key][ind] += 1
                            break
        #%%
print(cty)

{'A': {'TMAX': 245, 'TMIN': 523, 'TAVG': 70, 'PRCP': 561, 'SNWD': 0}, 'B': {'TMAX': 0, 'TMIN': 0, 'TAVG': 70, 'PRCP': 0, 'SNWD': 0}, 'C': {'TMAX': 239, 'TMIN': 471, 'TAVG': 129, 'PRCP': 207, 'SNWD': 0}}


In [46]:
data.isnull().sum()


Unnamed: 0     0
DATE           0
LATITUDE_A     0
LONGITUDE_A    0
ELEVATION_A    0
PRCP_A         0
SNWD_A         0
TMAX_A         0
TMIN_A         0
TAVG_A         0
LATITUDE_B     0
LONGITUDE_B    0
ELEVATION_B    0
PRCP_B         0
SNWD_B         0
TMAX_B         0
TMIN_B         0
TAVG_B         0
LATITUDE_C     0
LONGITUDE_C    0
ELEVATION_C    0
PRCP_C         0
SNWD_C         0
TMAX_C         0
TMIN_C         0
TAVG_C         0
LATITUDE       0
LONGITUDE      0
TAVG           0
dtype: int64

In [47]:
X=data.drop(['TAVG'],axis=1)
y=data['TAVG']
X=X.drop(['DATE'],axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [48]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

In [49]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [50]:
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [51]:
class weather(datum.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
train_dataset = weather(X_train, y_train)
test_dataset = weather(X_test, y_test)
train_loader = datum.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = datum.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim

class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5=# Final layer with no activation for regression output
        
        # Activation function
        self.relu = nn.ReLU()
        
        # Dropout layer with 50% probability
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # Forward pass with ReLU activations and Dropout
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)  # No activation in the final layer for regression
        return x

In [53]:
input_dim = X_train.shape[1]
model = Model(input_dim)

# Use MSELoss for regression tasks
criterion = nn.MSELoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [54]:
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/10], Loss: 252.9440
Epoch [2/10], Loss: 247.1804
Epoch [3/10], Loss: 231.2081
Epoch [4/10], Loss: 203.2013
Epoch [5/10], Loss: 148.3006
Epoch [6/10], Loss: 83.1906
Epoch [7/10], Loss: 68.1281
Epoch [8/10], Loss: 70.7547
Epoch [9/10], Loss: 55.5308
Epoch [10/10], Loss: 54.3724


In [55]:
model.eval()
correct = 0
total = 0
threshold = 0.5  # Experiment with different thresholds
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs > threshold).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 0.82%


In [56]:
testdf=pd.read_csv('test.csv')
testdf.drop(['DATE'],axis=1,inplace=True)
cty = {'A': {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}, 'B': {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}, 'C': {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}}
# cty = {'A': 0, 'B': 0, 'C': 0}
# fld = {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}
# cty = {i: fld for i, x in cty.items()}
# print(cty)
for x, dat in enumerate(testdf.itertuples()):
    for key, val in cty.items():
        for ind, tvl in val.items():
            rem = list(val.keys())
            rem.remove(ind)
            if pd.isna(getattr(dat, f'{ind}_{key}')):
                v1, v2 = getattr(dat, f'{rem[0]}_{key}'), getattr(dat, f'{rem[1]}_{key}')
                if not pd.isna(v1) and not pd.isna(v2) and ind != 'PRCP' and ind != 'SNWD':
                    testdf.loc[x, f'{ind}_{key}'] = ((2 * v2) - v1) if ind in ['TMAX', 'TMIN'] else ((v1 + v2) / 2)
                    cty[key][ind] += 1
                else:
                    rec = list(cty.keys())
                    rec.remove(key)
                    for ciy in rec:
                        if not pd.isna(getattr(dat, f'{ind}_{ciy}')):
                            testdf.loc[x, f'{ind}_{key}'] = float(getattr(dat, f'{ind}_{ciy}'))
                            cty[key][ind] += 1
                            break
        #%%
print(cty)


{'A': {'TMAX': 61, 'TMIN': 145, 'TAVG': 0, 'PRCP': 133, 'SNWD': 0}, 'B': {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}, 'C': {'TMAX': 52, 'TMIN': 108, 'TAVG': 6, 'PRCP': 45, 'SNWD': 0}}


In [57]:
import torch
import numpy as np

def predict(user_input):
    # Scale the input and convert it to a tensor
    user_input = scaler.transform(user_input)  # Scale the input data
    user_input = torch.tensor(user_input, dtype=torch.float32)  # Convert to PyTorch tensor
    
    # Set the model to evaluation mode
    model.eval()
    
    # Disable gradient calculation for prediction
    with torch.no_grad():
        # Pass the input through the model
        output = model(user_input)
    
    # If the output has multiple predictions, convert it to a list
    return output.squeeze().tolist()  # Convert tensor to list for multiple predictions

# Convert the test dataframe to a numpy array
user_input = testdf.to_numpy()  # Convert DataFrame to NumPy array

# Get predictions
prediction = predict(user_input)

# Print predictions
print(prediction)


[-2.469334602355957, 6.791647434234619, 10.338836669921875, 3.714327573776245, 10.048051834106445, -2.6468377113342285, 11.471733093261719, 3.5908071994781494, 4.403349876403809, 13.156148910522461, 4.283207893371582, 4.8258442878723145, 6.096195697784424, 16.200912475585938, 10.95086669921875, -3.169811248779297, 12.526456832885742, 18.830793380737305, -2.8743982315063477, -1.774031639099121, 16.021120071411133, -3.497422695159912, -2.498927354812622, -3.2556324005126953, 9.134977340698242, -4.635560035705566, -2.39042329788208, 18.176864624023438, 8.151437759399414, 2.646265983581543, 18.834169387817383, 13.821078300476074, 11.623830795288086, 14.181435585021973, -2.526798725128174, 10.298829078674316, 14.52481746673584, 15.71035099029541, -5.100314617156982, -4.984028339385986, 16.222999572753906, -4.394448757171631, -4.394420623779297, 13.057104110717773, 7.560232162475586, 19.934961318969727, 9.301198959350586, -5.249974727630615, -6.2671895027160645, -4.502742290496826, -3.305235

C:\Users\ragul\miniconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [58]:
df = pd.DataFrame({'TAVG':prediction}, index=testdf.index)

In [60]:
df.to_csv('ragulsub2.csv')